## GradientTapes and Eager Basics

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

## Gradient tapes

`Automatic differentiation` : 인풋에 대해서 Gradient 계산 
`TF`는 `tape` 안에서 시행된 연산을 모두 `기록합니다.`



In [20]:
x = tf.ones((2,2))

with tf.GradientTape() as t :
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)
    
dz_dx = t.gradient(z, x)

for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0

In [21]:
x = tf.ones((2,2))

with tf.GradientTape() as t :
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)
    
dz_dx = t.gradient(z, x)
dy_dx = t.gradient(y, x)  # 6.0
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

위에 보면 알 수 있듯이 에러가 발생한다. 

이유는 Gradient method를 호출하자 마자 리소스가 해체된다.

동일한 계산을 통해 여러 Gradient를 계산 하려면 영구 Gradient tape를 써야한다 `(persistent = True)`

그리고 나중에 제거하면 된다.

In [18]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x
    z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0
print(dz_dx , dy_dx)
del t  # Drop the reference to the tape

tf.Tensor(108.0, shape=(), dtype=float32) tf.Tensor(6.0, shape=(), dtype=float32)


## Recording Control Flow

* tapes는 시행했던 것을 기록하기 때문에 if문과 while 문을 쓸 수 있다.

In [26]:
def f(x, y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)
    return t.gradient(out, x) 

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


## Higher-order Gradients

* GradientTape 내부 연산은 자동 미분으로 기록되어진다.
* Gradient 가 계산 될 때 기록 되어진다.
* 그래서 결과적으로 정확히 높은 차수에 gradient도 잘 할 수 있다.


In [29]:
x = tf.Variable(1.0)

with tf.GradientTape() as t :
    with tf.GradientTape() as t2 :
        y = x*x*x
    
    dy_dx = t2.gradient(y,x)
    
d2y_dx2 = t.gradient(dy_dx , x)

print(dy_dx)
print(d2y_dx2)
assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0


tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


## Tensors

* `Tensor`는 다차원 array다. 유사하게  ndarray object , Tensor objects는 data type 과  shape를 가진다.
* 추가적으로 `Tensorflow`는 다양한 연산을 제공한다(tf.add, ...)
* 이러한 연산은 자동적으로 python type으로 바꿔준다.

In [34]:
print(tf.add(1,2))
print(tf.add([1,2],[3,4]))
print(tf.square(5))
print(tf.reduce_sum([1,2,3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))


x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)
(1, 2)
<dtype: 'int32'>


## Numpy 와 호환성

* numpy 와 tensorflow 전환은 간단하다.
    * TensorFlow 작업은 NumPy ndarrays를 Tensors로 자동 변환합니다.
    * NumPy 작업은 Tensors를 NumPy ndarrays로 자동 변환합니다.

* `.numpy()` 방법으로 바꿀 수 있다. 

In [37]:
import numpy as np

ndarray = np.ones([3,3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)

print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())



TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


# GPU acceleration
* 따로 GPU 와 CPU를 지정해주지 않으면 자동으로 정해준다.
* Tensors 전형적으로 메모리 기반으로 한다.


In [39]:
x = tf.random_uniform([3, 3])

print("Is there a GPU available: "),
#print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
#print(x.device.endswith('GPU:0'))

Is there a GPU available: 
Is the Tensor on GPU #0:  


(None,)

Device Names


In [42]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)

    result = time.time()-start
    
    print("10 loops: {:0.2f}ms".format(1000*result))


# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# Force execution on GPU #0 if available
if tf.test.is_gpu_available():
    with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random_uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 57.44ms
10 loops: 0.49ms


## DataSsets [tf.Data](https://www.tensorflow.org/guide/datasets)



In [44]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()


print(filename)

with open(filename, 'w') as f:
    f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

/tmp/tmpeuqdv0g3


## Apply transformations [Url](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

* Use the transformations functions like `map`, `batch`, `shuffle` etc. to apply transformations to the records of the dataset. 
* See the API documentation for tf.data.Dataset for details.

In [45]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

## Iterate

* Eager execution이 실행되면 , Dataset 객체는 반복을 지원한다고 한다.
* Tensorflow 그래프에서 DataSet 사용에 익숙하다면, `Dataset.make_one_shot_iterator()` 또는 `get_next()` 호출에 대한 `호출이 필요 없음`을 유의하십시오.

In [46]:
ds_tensors

<BatchDataset shapes: (?,), types: tf.int32>

In [47]:
print('Elements of ds_tensors:')
for x in ds_tensors:
    print(x)

print('\nElements in ds_file:')
for x in ds_file:
    print(x)

Elements of ds_tensors:
tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([16  9], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


## 다음에는 custom_layers 와 custom_training 도 살펴볼 예정이다.

## https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/notebooks